In [ ]:
#| default_exp visualization

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
#Weight & Biases
import wandb

#Yaml
from yaml import load, FullLoader

#Embeddings
from dvats.all import *
from tsai.data.preparation import prepare_forecasting_data
from tsai.data.validation import get_forecasting_splits
from fastcore.all import *

#Dimensionality reduction
from tsai.imports import *

#Clustering
import hdbscan
import utils.config as cfg_
import time
from dvats.dr import get_PCA_prjs, get_UMAP_prjs, get_TSNE_prjs


### Utilities for simpler code in 05_xai-lrp

In [2]:
#| export
def get_dataset(config, run_dr, config_dr, print_flag):
    # Botch to use artifacts offline
    artifacts_gettr = run_dr.use_artifact if config_dr.use_wandb else api.artifact
    enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')
    if print_flag: print (enc_artifact.name)
    ## TODO: This only works when you run it two timeS! WTF?
    try:
        enc_learner = enc_artifact.to_obj()
    except:
        enc_learner = enc_artifact.to_obj()
    #enc_artifact.metadata
    ## Restore artifact
    enc_logger = enc_artifact.logged_by()
    enc_artifact_train = artifacts_gettr(enc_logger.config['train_artifact'], type='dataset')
    if enc_logger.config['valid_artifact'] is not None:
        enc_artifact_valid = artifacts_gettr(enc_logger.config['valid_artifact'], type='dataset')
        if print_flag: print("enc_artifact_valid:", enc_artifact_valid.name)
    if print_flag: print("enc_artifact_train: ", enc_artifact_train.name)
    if config_dr.dr_artifact is not None:
        dr_artifact = artifacts_gettr(config_dr.dr_artifact)
    else:
        dr_artifact = enc_artifact_train
    if print_flag: print("DR artifact train: ", dr_artifact.name)
    df = dr_artifact.to_df()
    if print_flag: display(df.head())
    return df, dr_artifact, enc_artifact, enc_learner

In [ ]:
#| export
def umap_parameters(config_dr, config):
    umap_params_cpu = {
        'n_neighbors' : config_dr.n_neighbors,
        'min_dist' : config_dr.min_dist,
        'random_state': np.uint64(822569775), 
        'metric': config_dr.metric,
        #'a': 1.5769434601962196,
        #'b': 0.8950608779914887,
        #'metric_kwds': {'p': 2}, #No debería ser necesario, just in case
        #'output_metric': 'euclidean',
        'verbose': 4,
        #'n_epochs': 200
    }
    umap_params_gpu = {
        'n_neighbors' : config_dr.n_neighbors,
        'min_dist' : config_dr.min_dist,
        'random_state': np.uint64(1234), 
        'metric': config_dr.metric,
        'a': 1.5769434601962196,
        'b': 0.8950608779914887,
        'target_metric': 'euclidean',
        'target_n_neighbors': config_dr.n_neighbors,
        'verbose': 4, #6, #CUML_LEVEL_TRACE
        'n_epochs': 200*3*2,
        'init': 'random',
        'hash_input': True
    }
    if config.cpu_flag: 
        umap_params = umap_params_cpu
    else:
        umap_params = umap_params_gpu
    return umap_params

In [12]:
#| export
def get_prjs(embs_no_nan, print_flag = False):
    prjs_pca = get_PCA_prjs(
        X    = embs_no_nan, 
        cpu  = False,  
        print_flag = print_flag, 
        **umap_params
    )
    if print_flag: 
        print(prjs_pca.shape)
    prjs_umap = get_UMAP_prjs(
        input_data = prjs_pca, 
        cpu =  config.cpu_flag, #config_dr.cpu, 
        print_flag = print_flag,         
        **umap_params
    )
    if print_flag: prjs_umap.shape
    return prjs